# ERA5-Land Hourly Data

ERA5-Land is a high-resolution reanalysis dataset that provides a consistent and detailed view of land variables over several decades, combining model data with atmospheric forcing from ERA5 to ensure accuracy. By correcting input variables for altitude differences and leveraging indirect observational influences, it offers enhanced precision for land surface applications like flood and drought forecasting. Despite some inherent uncertainties, ERA5-Land's extensive temporal and spatial resolution makes it a valuable resource for decision-making and environmental analysis.

**Information on Dataset:**
* Source: [ERA5-Land Hourly Data](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land?tab=overview')
* Author:
* Notebook Version: 1.1 (Updated: December 09, 2024)

## 1. Specifying the paths and working directories

In [1]:
import os

''' ---- Hier die Verzeichnisse angeben ---- '''
download_folder = r".\data\era5-land-hourly-data\download"
working_folder = r".\data\era5-land-hourly-data\working"
geotiff_folder = r".\data\era5-land-hourly-data\geotiff"
csv_folder = r".\data\era5-land-hourly-data\csv"
output_folder = r".\data\era5-land-hourly-data\output"
''' ----- Ende der Eingaben ---- '''

os.makedirs(download_folder, exist_ok=True)
os.makedirs(working_folder, exist_ok=True)
os.makedirs(geotiff_folder, exist_ok=True)
os.makedirs(csv_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

## 2. Download and Extract Dataset

### 2.1 Authentication

In [2]:
import cdsapi

def main():
    api_key = "fdae60fd-35d4-436f-825c-c63fedab94a4"
    api_url = "https://cds.climate.copernicus.eu/api"
    client = cdsapi.Client(url=api_url, key=api_key)
    return client

### 2.2 Request Definition and Download

#### 1. Create list of variables

The list of variables are available at [ERA5-Land Hourly](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land?tab=download)

*Please check and modify the predefined variable name if any issue arises with variable name.* 

In [3]:
# Variable Groups
var_group_temperature = [
    "2m_dewpoint_temperature",
    "2m_temperature",
    "skin_temperature",
    "soil_temperature_level_1",
    "soil_temperature_level_2",
    "soil_temperature_level_3",
    "soil_temperature_level_4"]

var_group_lake = [
    "lake_bottom_temperature",
    "lake_ice_depth",
    "lake_ice_temperature",
    "lake_mix_layer_depth",
    "lake_mix_layer_temperature",
    "lake_shape_factor",
    "lake_total_layer_temperature"]

var_group_snow = [
    "snow_albedo",
    "snow_cover",
    "snow_density",
    "snow_depth",
    "snow_depth_water_equivalent",
    "snowfall",
    "snowmelt",
    "temperature_of_snow_layer"]

var_group_soil_water = [
    "skin_reservoir_content",
    "volumetric_soil_water_layer_1",
    "volumetric_soil_water_layer_2",
    "volumetric_soil_water_layer_3",
    "volumetric_soil_water_layer_4"]

var_group_radiation_and_heat = [
    "forecast_albedo",
    "surface_latent_heat_flux",
    "surface_net_solar_radiation",
    "surface_net_thermal_radiation",
    "surface_sensible_heat_flux",
    "surface_solar_radiation_downwards",
    "surface_thermal_radiation_downwards"]

var_group_evaporation_and_runoff = [
    "evaporation_from_bare_soil",
    "evaporation_from_open_water_surfaces_excluding_oceans",
    "evaporation_from_the_top_of_canopy",
    "evaporation_from_vegetation_transpiration",
    "potential_evaporation",
    "runoff",
    "snow_evaporation",
    "sub_surface_runoff",
    "surface_runoff",
    "total_evaporation"]

var_group_wind_pressure_and_precipitation = [
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "surface_pressure",
    "total_precipitation"]

var_group_vegetation = [
    "leaf_area_index_high_vegetation",
    "leaf_area_index_low_vegetation"]

# List of variable group
var_group_list = ['var_group_temperature',
                  'var_group_lake',
                  'var_group_soil_water',
                  'var_group_radiation_and_heat',
                  'var_group_evaporation_and_runoff',
                  'var_group_wind_pressure_and_precipitation',
                  'var_group_vegetation']

#### 2. Define request parameters for download

In [4]:
import ipywidgets as widgets

selected_variable_group = widgets.Dropdown(
    options=var_group_list,
    value=var_group_list[0],
    description="Select a variable group",
    style=dict(description_width='initial'),
    layout=widgets.Layout(width='50%'),
)

selected_variable_group

Dropdown(description='Select a variable group', layout=Layout(width='50%'), options=('var_group_temperature', …

In [5]:
current_variable_group = globals().get(selected_variable_group.value)

selected_variable = widgets.Dropdown(
    options=current_variable_group,
    value=current_variable_group[0],
    description="Select the variable of interest",
    style=dict(description_width='initial'),
    layout=widgets.Layout(width='50%'),
)

selected_variable

Dropdown(description='Select the variable of interest', layout=Layout(width='50%'), options=('2m_dewpoint_temp…

#### 3. Define Bounding Box Extents (Bbox) for the Dataset

In [6]:
# Define additional request fields to ensure the request stays within the file size limit.
# These coordinates were obtained using the BBox Extractor tool: https://str-ucture.github.io/bbox-extractor/

bbox_wgs84_deutschland = [56.0, 5.8, 47.2, 15.0]
bbox_wgs84_konstanz = [47.9, 8.9, 47.6, 9.3]

In [7]:
from datetime import datetime

selected_year = widgets.Dropdown(
    options=[str(year) for year in range(1950, datetime.now().year + 1)],
    value=str(datetime.now().year),
    description="Select the year for downloading data:",
    disabled=False,
    style=dict(description_width='initial'),
    layout=widgets.Layout(width='50%'),
)

selected_year

Dropdown(description='Select the year for downloading data:', index=74, layout=Layout(width='50%'), options=('…

#### 4. Define request parameters

In [8]:
dataset = "reanalysis-era5-land"
request = {
    "variable": selected_variable.value,
    "year": selected_year.value,
    "month": [str(month) for month in range(13)],
    "day": [str(day) for day in range(32)],
    "time": [f"{hour:02d}:00" for hour in range(24)],
    "data_format": "netcdf",
    "download_format": "unarchived",
    "area": bbox_wgs84_deutschland
}

In [9]:
# Uncomment and run this cell to download the dataset:

def main_retrieve():
    dataset_filename = f"{dataset}.zip"
    dataset_filepath = os.path.join(download_folder, dataset_filename)

    # Download the dataset only if the dataset has not been downloaded before
    if not os.path.isfile(dataset_filepath):
        # Download the dataset with the defined request parameters
        client.retrieve(dataset, request, dataset_filepath)
    else:
        print("Dataset already downloaded.")

if __name__ == "__main__":
    client = main()
    main_retrieve()

2024-12-17 10:15:00,210 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.


2024-12-17 10:15:00,212 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.


2024-12-17 10:15:00,213 INFO [2024-09-16T00:00:00] Remember that you need to have an ECMWF account to use the new CDS. **Your old CDS credentials will not work in new CDS!**


2024-12-17 10:15:00,214 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.


Dataset already downloaded.


### 2.3 Extract the Zip folder

In [10]:
import zipfile

dataset_filename = f"{dataset}.zip"
dataset_filepath = os.path.join(download_folder, dataset_filename)
extract_folder = working_folder

# Extract the zip file
try:
    os.makedirs(extract_folder, exist_ok=True)
    
    if not os.listdir(extract_folder):
        with zipfile.ZipFile(dataset_filepath, 'r') as zip_ref:
            zip_ref.extractall(extract_folder)
            print(f"Successfully extracted files to: {extract_folder}")
    else:
        print("Folder is not empty. Skipping extraction.")
except FileNotFoundError:
    print(f"Error: The file {dataset_filepath} was not found.")
except zipfile.BadZipFile:
    print(f"Error: The file {dataset_filepath} is not a valid zip file.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Error: The file .\data\era5-land-hourly-data\download\reanalysis-era5-land.zip is not a valid zip file.
